## Análisis - Ligue 1 temporada 2015/2016 (masculina)

Vamos a hacer un análisis de las distintas competiciones (las cinco grandes ligas durante la temporada 2015-2016 y de todos los partidos en general) para poder entender que características son más importantes a la hora de tomar la decisión por parte del modelo de ML para decidir si el equipo va a ganar, perder o empatar el partido.

En este notebook en cuestión, vamos a analizar la Ligue 1 en la temporada 2015/2016 (masculina).

Cargamos los datos de los partidos sobre la competición para entrenar el modelo y poder hacer el estudio de explicabilidad.

In [1]:
import os
import pandas as pd


competition_name = "Ligue 1"
competition_gender = "male"
season_name = "2015/2016"
season_name_for_filename = "2015_2016"

output_dir = "data/reduced/"
filename = f"{competition_name}({season_name_for_filename}_{competition_gender})_reduced.csv"
output_path = os.path.join(output_dir, filename)

if os.path.exists(output_path):
    print(f"Matches from the competition loaded from {output_path}")
    matches_in_Ligue1 = pd.read_csv(output_path)
else:
    print("You don't have data from the competition yet. Please run the notebook '0_datasets_building.ipynb' first.")

Matches from the competition loaded from data/reduced/Ligue 1(2015_2016_male)_reduced.csv


In [2]:
matches_in_Ligue1.head(5)

,match_id,shots_on_target_ratio_home,shots_on_target_ratio_away,average_shots_on_target_distance_home,average_shots_on_target_distance_away,shots_inside_area_ratio_home,shots_inside_area_ratio_away,pass_success_ratio_home,pass_success_ratio_away,cross_success_ratio_home,...,percentage_shots_under_pressure_home,percentage_shots_inside_area_under_pressure_home,percentage_passes_under_pressure_home,percentage_passes_inside_area_under_pressure_home,percentage_set_piece_shots_home,percentage_set_piece_shots_inside_area_home,percentage_substitutions_home,percentage_tactical_substitutions_home,percentage_tactical_changes_home,percentage_formation_changes_home
0,3829421,0.333333,0.166667,25.129294,12.803802,0.555556,0.583333,0.763676,0.809074,0.307692,...,0.333333,0.333333,0.566265,1.0,0.555556,0.666667,0.5,0.50,0.750000,0.666667
1,3829420,0.307692,0.428571,20.107446,19.128651,0.230769,0.714286,0.747412,0.783058,0.500000,...,0.375000,0.166667,0.565217,0.5,0.500000,0.222222,0.5,0.40,0.444444,0.750000
2,3829413,0.166667,0.333333,21.737917,19.467647,0.333333,0.833333,0.798755,0.829213,0.200000,...,0.000000,0.000000,0.520000,0.0,0.666667,0.500000,0.5,0.50,0.500000,0.000000
3,3829416,0.333333,0.352941,12.209684,15.420536,0.666667,0.764706,0.741688,0.810964,0.250000,...,0.250000,0.285714,0.514085,0.0,0.384615,0.333333,0.5,0.25,0.625000,1.000000
4,3829422,0.227273,0.500000,20.112628,20.723704,0.500000,0.500000,0.876560,0.786082,0.142857,...,0.444444,0.600000,0.684564,1.0,0.875000,0.875000,0.5,0.50,0.500000,0.500000


Una vez ya tenemos los datos cargados, entrenamos el mejor modelo (seleccionado tras la experimentación) y mostramos sus resultados.

In [3]:
from src.analysis import ligue1_best_model

best_model_Ligue1, evaluation_metrics, X_train_Ligue1, X_test_Ligue1, encoder_Ligue1, match_ids_test = ligue1_best_model(matches_in_Ligue1)
print(f"The best model for Ligue 1 is {best_model_Ligue1}")
print(f"The evaluation metrics for the best model are:")
evaluation_metrics.head()

The best model for Ligue 1 is RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=2, n_estimators=60, random_state=42)
The evaluation metrics for the best model are:


,Train Accuracy,Test Accuracy,Precision Macro,Precision Weighted,Recall Macro,Recall Weighted,F1 Macro,F1 Weighted
Random Forest,0.774086,0.618421,0.591787,0.60164,0.589015,0.618421,0.581168,0.601067


Una vez ya tenemos el mejor modelo entrenado comenzamos el análisis de explicabilidad.